<a href="https://colab.research.google.com/github/ManasChandan/Pneumonia-Detection/blob/master/Pneumonia_DenseNet_%2C_ResNet_%2C_Basics_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"manascbehera123","key":"19e7fe4611ac8913c29a87790be44bca"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

chest-xray-pneumonia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!pip install tensorflow==2.2.0

In [ ]:
!unzip -qq -o chest-xray-pneumonia.zip

In [ ]:
train_path = 'chest_xray/train'
valid_path = 'chest_xray/val'
test_path = 'chest_xray/test'

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping
from tensorflow.keras.layers import Dense , Flatten , Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [ ]:
train_images = glob(train_path + '/*/*.jpeg')
valid_images = glob(valid_path + '/*/*.jpeg')
test_images = glob(test_path + '/*/*.jpeg')

In [ ]:
Image_Size = [224 , 224]

In [ ]:
transfer_model = DenseNet121(include_top= False , weights= 'imagenet' , input_shape= Image_Size + [3])

In [ ]:
for layer in transfer_model.layers:
  layer.trainable = False

In [ ]:
from tensorflow.keras.layers import Dropout

In [ ]:
x = Flatten()(transfer_model.output)
x = Dense(2 , activation='softmax')(x)

model = Model(inputs=transfer_model.input , outputs = x)

In [ ]:
model.summary()

In [ ]:
train_datagen1=ImageDataGenerator(rescale=1.0/255,
                                 rotation_range=30,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range=0.2,
                                 )

val_datagen1=ImageDataGenerator(rescale=1.0/255)

test_datagen1=ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_generator = train_datagen1.flow_from_directory(
    train_path , 
    target_size = Image_Size ,
    class_mode = 'categorical' ,
    batch_size = 32
)
test_generator = test_datagen1.flow_from_directory(
    test_path , 
    target_size = Image_Size ,
    class_mode = 'categorical' ,
    batch_size = 32
)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='Auto')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')

In [ ]:
model.compile(
    optimizer = 'adam' , 
    loss = 'binary_crossentropy' ,
    metrics = ['accuracy','Precision','Recall','FalseNegatives','FalsePositives','TrueNegatives','TruePositives']
)

In [ ]:
hist = model.fit_generator(
           train_generator, 
           epochs=30, validation_data=test_generator, 
           callbacks=[lr_reduce])

Epoch 1/30
163/163 [==============================] - 103s 632ms/step - loss: 3.9431 - accuracy: 0.7429 - precision: 0.7429 - recall: 0.7429 - false_negatives: 1341.0000 - false_positives: 1341.0000 - true_negatives: 3875.0000 - true_positives: 3875.0000 - val_loss: 5.7514 - val_accuracy: 0.6250 - val_precision: 0.6250 - val_recall: 0.6250 - val_false_negatives: 234.0000 - val_false_positives: 234.0000 - val_true_negatives: 390.0000 - val_true_positives: 390.0000 - lr: 0.0010
Epoch 2/30
163/163 [==============================] - 101s 620ms/step - loss: 3.9431 - accuracy: 0.7429 - precision: 0.7429 - recall: 0.7429 - false_negatives: 1341.0000 - false_positives: 1341.0000 - true_negatives: 3875.0000 - true_positives: 3875.0000 - val_loss: 5.7514 - val_accuracy: 0.6250 - val_precision: 0.6250 - val_recall: 0.6250 - val_false_negatives: 234.0000 - val_false_positives: 234.0000 - val_true_negatives: 390.0000 - val_true_positives: 390.0000 - lr: 0.0010
Epoch 3/30
163/163 [==================

KeyboardInterrupt: ignored

In [ ]:
model.save('pneumonia-densenet(small).h5')

In [ ]:
plt.plot(hist.history['loss'], label='train loss')
plt.plot(hist.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(hist.history['accuracy'], label='train acc')
plt.plot(hist.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
import cv2
import numpy as np

In [ ]:
import os

In [ ]:
from tensorflow.keras.models import load_model
model_new = load_model('pneumonia-modelnew.h5')

In [ ]:
normal_photos = test_path + "/NORMAL"
real_values = []
predicted_values = []
for img in os.listdir(normal_photos):
  real_values.append(0)
  path = os.path.join(normal_photos,img)
  imgnew = cv2.imread(path)
  imgnew = cv2.resize(imgnew, (224,224))
  imgnew = np.expand_dims(imgnew ,axis = 0)
  imgnew = imgnew / 255.0
  ans = model.predict(imgnew).argmax(axis=1)
  predicted_values.append(ans[0])

In [ ]:
pneumonia_photos = test_path + "/PNEUMONIA"
for img in os.listdir(pneumonia_photos):
  real_values.append(1)
  path = os.path.join(pneumonia_photos,img)
  imgnew = cv2.imread(path)
  imgnew = cv2.resize(imgnew, (224,224))
  imgnew = np.expand_dims(imgnew ,axis = 0)
  imgnew = imgnew / 255.0
  ans = model.predict(imgnew).argmax(axis=1)
  predicted_values.append(ans[0])

In [ ]:
print(len(real_values),len(predicted_values))

In [ ]:
from sklearn.metrics import classification_report , roc_curve , confusion_matrix

In [ ]:
print(classification_report(real_values , predicted_values))

In [ ]:
print(confusion_matrix(real_values,predicted_values))

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
r_auc = roc_auc_score(real_values , predicted_values)
print(r_auc)

In [ ]:
r_fpr, r_tpr, _ = roc_curve(real_values , predicted_values)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(r_fpr , r_tpr)
# Title
plt.title('ROC Plot')
# Axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# Show legend
plt.legend() # 
# Show plot
plt.show()

In [ ]:
sensitivity = (382/(382+8))
print(sensitivity)

In [ ]:
specificity = (188/(46+188))

In [ ]:
print(specificity)